# 🧰 Week 5-6, Notebook 2: A Deep Dive into Hugging Face Pipelines

**Module:** LLMs, Prompt Engineering & RAG  
**Project:** Build the Knowledge Core for the Manufacturing Copilot

---

In the previous notebook, we introduced the core concepts of Large Language Models. Now, it's time to get hands-on and explore the easiest way to use them: the Hugging Face `pipeline` function.

**What is a Pipeline?**

A pipeline is a high-level, easy-to-use API that handles all the complexity of model loading, tokenization, inference, and decoding for a specific task. Instead of writing dozens of lines of code to set up a model, you can often achieve your goal in just a few lines with a pipeline.

This notebook provides a practical, manufacturing-focused tour of the most important pipelines, framing each one as a tool to solve a real-world problem for our **Manufacturing Copilot**.

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:

1.  **Map Scenarios to Pipelines:** Identify the correct Hugging Face pipeline for common NLP tasks like summarization, translation, and classification.
2.  **Customize Pipeline Behavior:** Control model selection, batch processing, and hardware allocation (CPU/GPU) to optimize for speed or accuracy.
3.  **Chain Pipelines:** Combine multiple pipelines to create sophisticated workflows (e.g., translate a document, then summarize it).
4.  **Integrate with Applications:** Understand the data structures (inputs and outputs) for each pipeline, enabling you to integrate them into your own Python applications.

## 🏭 The Right Tool for the Job: A Manufacturing Pipeline Selector

The Hugging Face Hub hosts tens of thousands of models, each specialized for different tasks. The `pipeline` function automatically selects a reasonable default model for a given task, but you can easily override it. Here’s a guide to the most useful pipelines for our manufacturing context.

| Manufacturing Goal | Recommended Pipeline | Example Use Case |
| :--- | :--- | :--- |
| **Gauge Operator Sentiment** | `sentiment-analysis` | Analyze logbook entry: 'Shift B reported a strong chemical odor near CNC-7.' |
| **Summarize Shift Logs** | `summarization` | Condense a multi-paragraph shift handover report into key bullet points. |
| **Translate Supplier Notices** | `translation` | Translate a German safety bulletin for a new piece of equipment into English. |
| **Retrieve SOP Steps** | `question-answering` | Find the correct torque specification from a 500-page maintenance manual. |
| **Draft Downtime Reports** | `text-generation` | Generate a preliminary incident report from a structured alert. |
| **Classify New Tickets** | `zero-shot-classification`| Categorize a new machine failure mode without needing a pre-existing label. |
| **Extract Structured Data** | `token-classification` (NER) | Pull part numbers, asset IDs, and dates from an unstructured work order. |
| **Fill in Missing Data** | `fill-mask` | Suggest a plausible value for a corrupted or missing sensor reading in a log file. |

## ⚙️ Environment Setup and Quick Check

Before we dive in, let's make sure our environment is set up correctly and check the versions of our key libraries. We'll also define a `device` variable to automatically leverage a GPU if one is available, which will significantly speed up inference.

import torch
import transformers

# Print library versions for reproducibility
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")

# Check for GPU availability and set the device
if torch.cuda.is_available():
    device = 0  # Use the first GPU (device 0)
    print(f"GPU is available: {torch.cuda.get_device_name(device)}")
else:
    device = -1  # Use CPU
    print("GPU not available, using CPU.")

## Task 1: Sentiment Analysis

**Use Case:** Automatically monitor operator feedback from a digital logbook. This can help you gauge morale, identify recurring safety concerns, or flag urgent issues that require immediate attention.

A sentiment analysis pipeline classifies a piece of text as positive, negative, or neutral. It's a quick and powerful way to get a high-level understanding of large volumes of unstructured text.

In [ ]:
# Hands-On: Sentiment Analysis
from transformers import pipeline

# Example feedback entries from a factory floor logbook
feedback_entries = [
    "The refurbished compressor is running silently and our energy draw has dropped by 3%. Great work by the maintenance team!",
    "The paint booth filter change was skipped again during the night shift. We had to scrap 18 panels due to overspray contamination.",
    "The new safety guards on the stamping press are working as expected."
]

# Initialize the sentiment analysis pipeline
# We can specify a model fine-tuned on text similar to our use case.
# 'cardiffnlp/twitter-roberta-base-sentiment' is trained on informal text and works well here.
sentiment_pipeline = pipeline(
    task='sentiment-analysis',
    model='cardiffnlp/twitter-roberta-base-sentiment',
    device=device  # Use GPU if available
)

# Run the feedback through the pipeline
results = sentiment_pipeline(feedback_entries)

# Print the results in a readable format
for text, result in zip(feedback_entries, results):
    # The model returns 'LABEL_0' (Negative), 'LABEL_1' (Neutral), 'LABEL_2' (Positive)
    # We can map these to more descriptive names.
    label_map = {'LABEL_0': 'Negative', 'LABEL_1': 'Neutral', 'LABEL_2': 'Positive'}
    print(f"Feedback: '{text[:60]}...'")
    print(f"  -> Sentiment: {label_map.get(result['label'], 'Unknown')} (Score: {result['score']:.3f})\n")

In [ ]:
# Hands-On: Summarization
from transformers import pipeline

# A detailed shift log entry
shift_log = """
Shift Supervisor: Maria Rodriguez
Date: 2024-07-16, Night Shift (22:00 - 06:00)

Summary of Operations:
Line 3 experienced intermittent stoppages between 01:15 and 02:45, traced to a faulty sensor (ID: S-45B) on the conveyor belt. Maintenance was notified, and Technician David Chen bypassed the sensor at 03:00 to restore production. A permanent fix is scheduled for the next maintenance window. Total downtime was approximately 90 minutes, with a loss of 350 units.
Line 1 and 2 operated at 98% efficiency with no major issues.

Quality Control:
QC team flagged a batch of 50 units from Line 3 (Batch ID: B-7891) produced just before the downtime for potential defects. They are currently under inspection.

Safety:
A minor coolant spill was reported near CNC machine #5 at 04:30. The area was cleaned and cordoned off per protocol. No injuries were reported.
"""

# Initialize the summarization pipeline
# 'facebook/bart-large-cnn' is a great model for summarizing news-like articles and reports.
summarizer = pipeline(
    task="summarization",
    model="facebook/bart-large-cnn",
    device=device
)

# Generate the summary
summary = summarizer(
    shift_log,
    max_length=100,  # The maximum length of the summary
    min_length=30,   # The minimum length of the summary
    do_sample=False
)

print("Original Log Length:", len(shift_log))
print("Summary Length:", len(summary[0]['summary_text']))
print("\nGenerated Summary:\n", summary[0]['summary_text'])

In [ ]:
## Task 3: Translation

**Use Case:** Your company sources equipment from around the world. A new CNC machine arrives with its technical documentation and safety warnings in German. A translation pipeline can instantly make this critical information accessible to your English-speaking maintenance and safety teams.

Hugging Face provides access to some of the world's most powerful translation models, originally developed by the University of Helsinki. These models are available for a vast number of language pairs.

In [ ]:
# Hands-On: Translation
from transformers import pipeline

# A safety warning in German
german_text = "Achtung: Die Maschine darf nur von autorisiertem Personal bedient werden. Vor Wartungsarbeiten Stromzufuhr unterbrechen."

# Initialize the translation pipeline
# Model names follow a pattern: "Helsinki-NLP/opus-mt-{source_language}-{target_language}"
translator = pipeline(
    task="translation_de_to_en",
    model="Helsinki-NLP/opus-mt-de-en",
    device=device
)

# Perform the translation
english_text = translator(german_text)

print(f"Original (German): {german_text}")
print(f"Translated (English): {english_text[0]['translation_text']}")

In [ ]:
## Task 4: Question Answering

**Use Case:** A technician needs to find a specific piece of information—like a torque specification or a pressure setting—buried deep within a long PDF maintenance manual. A question-answering (QA) system can instantly find the answer, saving valuable time and reducing the risk of errors.

QA models work by taking a `question` and a `context` (the text to search within) and extracting the span of text that best answers the question. This is a form of **extractive QA**.

In [ ]:
## Task 2: Summarization

**Use Case:** Condense long, detailed shift handover logs or maintenance reports into a few key bullet points. This allows supervisors and engineers to quickly grasp the most critical information without reading through pages of text.

Summarization models are trained to take a long document and generate a shorter, coherent version that captures the main ideas.